* TECH CHALLENGE - FASE 03: FINE-TUNING AMAZON PRODUCTS
* Versão Simplificada com OpenAI API

* 1: INSTALAÇÃO E CONFIGURAÇÃO

In [1]:
!pip install -q openai pandas gradio

In [2]:
import json
import pandas as pd
import openai
from openai import OpenAI
import gradio as gr
import getpass

In [ ]:
client = OpenAI(api_key="")

In [4]:
# Testar conexão
try:
    models = client.models.list()
    print("Conexão com OpenAI estabelecida!")
    print("Modelos disponíveis para fine-tuning:", [m.id for m in models.data if 'gpt-3.5-turbo' in m.id][:3])
except Exception as e:
    print(f"Erro na conexão: {e}")

Conexão com OpenAI estabelecida!
Modelos disponíveis para fine-tuning: ['gpt-3.5-turbo', 'gpt-3.5-turbo-instruct', 'gpt-3.5-turbo-instruct-0914']


* 2: UPLOAD E ANÁLISE DO DATASET

In [ ]:
!gunzip trn.json.gz

In [ ]:
# Verificar se arquivo existe ou criar exemplo
try:
    with open('/content/trn.json', 'r') as f:
        test_read = f.read(1)
    print("Dataset encontrado!")
except FileNotFoundError:
    print("\nArquivo não existe!")

Dataset encontrado!


In [ ]:
# Analisar dataset
def analyze_dataset():
    print("\nANÁLISE DO DATASET:")

    data = []
    with open('/content/trn.json', 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= 10:  # Analisar primeiros 10
                break
            try:
                item = json.loads(line)
                data.append(item)
            except:
                continue

    if data:
        print(f"{len(data)} exemplos carregados")
        print(f"Campos: {list(data[0].keys())}")

        # Mostrar exemplo
        print(f"\nExemplo:")
        print(f"Título: {data[0]['title']}")
        print(f"Descrição: {data[0]['content'][:100]}...")

        return data
    else:
        print("Erro ao carregar dados")
        return []

In [ ]:
sample_data = analyze_dataset()


ANÁLISE DO DATASET:
10 exemplos carregados
Campos: ['uid', 'title', 'content', 'target_ind', 'target_rel']

Exemplo:
Título: Girls Ballet Tutu Neon Pink
Descrição: High quality 3 layer ballet tutu. 12 inches in length...


* 3: PREPARAÇÃO DOS DADOS PARA FINE-TUNING

In [ ]:
def prepare_openai_dataset(max_examples=50):
    """Prepara dados no formato da OpenAI para fine-tuning"""

    print(f"Preparando {max_examples} exemplos para fine-tuning...")

    training_data = []

    with open('/content/trn.json', 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i >= max_examples:
                break

            try:
                item = json.loads(line)
                title = item.get('title', '').strip()
                content = item.get('content', '').strip()

                if not title or not content:
                    continue

                # Criar exemplos de treinamento
                questions = [
                    f"Me fale sobre: {title}",
                    f"Quais as características de: {title}",
                    f"Descreva este produto: {title}"
                ]

                for question in questions[:2]:  # 2 exemplos por produto
                    training_example = {
                        "messages": [
                            {
                                "role": "system",
                                "content": "Você é um assistente especializado em produtos da Amazon. Forneça informações detalhadas sobre produtos baseado nas descrições fornecidas."
                            },
                            {
                                "role": "user",
                                "content": question
                            },
                            {
                                "role": "assistant",
                                "content": content
                            }
                        ]
                    }
                    training_data.append(training_example)

            except Exception as e:
                continue

    print(f"{len(training_data)} exemplos preparados")

    # Salvar em formato JSONL para OpenAI
    with open('/content/training_data.jsonl', 'w', encoding='utf-8') as f:
        for example in training_data:
            f.write(json.dumps(example, ensure_ascii=False) + '\n')

    print("Dados salvos em '/content/training_data.jsonl'")
    return training_data

In [ ]:
# Preparar dados
training_examples = prepare_openai_dataset(25)  # 25 produtos = ~50 exemplos

# Mostrar exemplo
print("\nExemplo de dados preparados:")
print(json.dumps(training_examples[0], indent=2, ensure_ascii=False))

Preparando 25 exemplos para fine-tuning...
22 exemplos preparados
Dados salvos em '/content/training_data.jsonl'

Exemplo de dados preparados:
{
  "messages": [
    {
      "role": "system",
      "content": "Você é um assistente especializado em produtos da Amazon. Forneça informações detalhadas sobre produtos baseado nas descrições fornecidas."
    },
    {
      "role": "user",
      "content": "Me fale sobre: Girls Ballet Tutu Neon Pink"
    },
    {
      "role": "assistant",
      "content": "High quality 3 layer ballet tutu. 12 inches in length"
    }
  ]
}


* 4: TESTE DO MODELO BASE (ANTES DO FINE-TUNING)

In [ ]:
def test_base_model():
    """Testa GPT-3.5 sem fine-tuning"""

    print("TESTE DO MODELO BASE (SEM FINE-TUNING)")

    test_questions = [
        "Me fale sobre fones de ouvido Sony WH-1000XM4",
        "Quais as características do Samsung Galaxy S21 Ultra",
        "Descreva o MacBook Air com M1"
    ]

    for question in test_questions:
        print(f"\nPergunta: {question}")

        try:
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "Você é um assistente de produtos."},
                    {"role": "user", "content": question}
                ],
                max_tokens=150,
                temperature=0.7
            )

            answer = response.choices[0].message.content
            print(f"Resposta BASE: {answer}")

        except Exception as e:
            print(f"Erro: {e}")

    print("\n" + "="*60)

In [ ]:
test_base_model()

TESTE DO MODELO BASE (SEM FINE-TUNING)

Pergunta: Me fale sobre fones de ouvido Sony WH-1000XM4
Resposta BASE: Os fones de ouvido Sony WH-1000XM4 são conhecidos por sua excelente qualidade de som e cancelamento de ruído líder de mercado. Eles oferecem uma experiência de áudio imersiva, com drivers de 40mm que proporcionam graves profundos e agudos cristalinos. 

Além disso, esses fones possuem tecnologia de cancelamento de ruído adaptável, que ajusta automaticamente a intensidade do cancelamento de acordo com o ambiente ao seu redor. Isso garante uma experiência auditiva tranquila, seja em um ambiente movimentado ou em um local mais silencioso.

Os Sony WH-1000XM4 também contam

Pergunta: Quais as características do Samsung Galaxy S21 Ultra
Resposta BASE: O Samsung Galaxy S21 Ultra é um smartphone topo de gama da Samsung, lançado em 2021. Algumas das características principais deste modelo incluem:

- Tela: Dynamic AMOLED de 6,8 polegadas com resolução de 3200 x 1440 pixels e taxa de a

* 5: UPLOAD DOS DADOS E FINE-TUNING

In [ ]:
def upload_and_finetune():
    """Faz upload dos dados e executa fine-tuning na OpenAI"""

    print("INICIANDO PROCESSO DE FINE-TUNING")

    try:
        # 1. Upload do arquivo de treinamento
        print("Fazendo upload dos dados de treinamento...")

        with open('/content/training_data.jsonl', 'rb') as f:
            training_file = client.files.create(
                file=f,
                purpose='fine-tune'
            )

        print(f"Arquivo enviado! ID: {training_file.id}")

        # 2. Criar job de fine-tuning
        print("Criando job de fine-tuning...")

        fine_tune_job = client.fine_tuning.jobs.create(
            training_file=training_file.id,
            model="gpt-3.5-turbo-1106",  # Modelo base para fine-tuning
            hyperparameters={
                "n_epochs": 3,  # Número de épocas
                "batch_size": 1,
                "learning_rate_multiplier": 0.1
            }
        )

        print(f"Job criado! ID: {fine_tune_job.id}")
        print(f"Status: {fine_tune_job.status}")

        # Salvar IDs importantes
        job_info = {
            "training_file_id": training_file.id,
            "job_id": fine_tune_job.id,
            "status": fine_tune_job.status
        }

        with open('/content/finetune_info.json', 'w') as f:
            json.dump(job_info, f, indent=2)

        print("Informações salvas em '/content/finetune_info.json'")
        return fine_tune_job

    except Exception as e:
        print(f"Erro no fine-tuning: {e}")
        return None

In [ ]:
finetune_job = upload_and_finetune()

INICIANDO PROCESSO DE FINE-TUNING
Fazendo upload dos dados de treinamento...
Arquivo enviado! ID: file-P8CfbxPRChTrLj4uvPUiwC
Criando job de fine-tuning...
Job criado! ID: ftjob-g6uQNtY3iHSWBSZhBuCdeU3a
Status: validating_files
Informações salvas em '/content/finetune_info.json'


* 6: MONITORAR PROGRESSO DO FINE-TUNING

In [ ]:
import time
from IPython.display import clear_output

def monitor_finetune_progress():
    """Monitora o progresso do fine-tuning automaticamente com progress bar"""

    try:
        # Carregar informações do job
        with open('/content/finetune_info.json', 'r') as f:
            job_info = json.load(f)

        job_id = job_info['job_id']
        print(f"Monitorando job: {job_id}")
        print("Verificando status automaticamente...")

        while True:
            try:
                # Verificar status
                job = client.fine_tuning.jobs.retrieve(job_id)

                # Limpar output anterior
                clear_output(wait=True)

                # Mostrar status atual
                print("=" * 60)
                print("MONITORAMENTO AUTOMÁTICO DO FINE-TUNING")
                print("=" * 60)
                print(f"Job ID: {job.id}")
                print(f"Status: {job.status}")
                print(f"Modelo base: {job.model}")

                # Progress bar visual
                if job.status == "running":
                    print("\nProcessando... ⏳")
                    print("█████████████████████░░░░░ 75% (estimado)")
                elif job.status == "validating":
                    print("\nValidando modelo... 🔍")
                    print("████████████████████████░ 95% (estimado)")
                elif job.status == "succeeded":
                    print("\nConcluído! ✅")
                    print("█████████████████████████ 100%")
                elif job.status == "failed":
                    print("\nFalhou! ❌")
                    print("Error:", job.error)
                    break

                # Se terminou com sucesso
                if job.finished_at and job.status == "succeeded":
                    print(f"\nConcluído em: {job.finished_at}")
                    print(f"Modelo fine-tuned: {job.fine_tuned_model}")

                    # Salvar modelo ID
                    job_info['fine_tuned_model'] = job.fine_tuned_model
                    job_info['status'] = 'succeeded'
                    with open('/content/finetune_info.json', 'w') as f:
                        json.dump(job_info, f, indent=2)

                    print("\nFINE-TUNING CONCLUÍDO COM SUCESSO!")
                    print("Agora você pode executar a próxima célula.")
                    break

                # Se falhou
                elif job.status == 'failed':
                    print(f"\nFalhou: {job.error}")
                    job_info['status'] = 'failed'
                    with open('/content/finetune_info.json', 'w') as f:
                        json.dump(job_info, f, indent=2)
                    break

                # Se ainda em progresso
                else:
                    current_time = datetime.now().strftime("%d/%m/%Y às %H:%M:%S")
                    print(f"\nStatus: {job.status}")
                    print(f"Última verificação: {current_time}")
                    print("Verificando novamente em 30 segundos...")
                    print("\n(Pressione Ctrl+C para parar o monitoramento)")
                    time.sleep(30)  # Esperar 30 segundos

            except KeyboardInterrupt:
                print("\n\nMonitoramento interrompido pelo usuário.")
                print("Execute esta célula novamente para continuar monitorando.")
                break
            except Exception as e:
                print(f"Erro na verificação: {e}")
                print("Tentando novamente em 30 segundos...")
                time.sleep(30)

        return job

    except Exception as e:
        print(f"Erro ao iniciar monitoramento: {e}")
        return None

In [ ]:
print("Iniciando monitoramento automático...")
final_job = monitor_finetune_progress()

MONITORAMENTO AUTOMÁTICO DO FINE-TUNING
Job ID: ftjob-g6uQNtY3iHSWBSZhBuCdeU3a
Status: succeeded
Modelo base: gpt-3.5-turbo-1106

Concluído! ✅
█████████████████████████ 100%

Concluído em: 1757341408
Modelo fine-tuned: ft:gpt-3.5-turbo-1106:personal::CDX4FhCa

FINE-TUNING CONCLUÍDO COM SUCESSO!
Agora você pode executar a próxima célula.


* 7: TESTE DO MODELO FINE-TUNED

In [5]:
def test_finetuned_model():
    """Testa o modelo após fine-tuning"""

    try:
        # Carregar informações do modelo
        with open('/content/finetune_info.json', 'r') as f:
            job_info = json.load(f)

        if 'fine_tuned_model' not in job_info:
            print("Modelo ainda não está pronto. Verifique o status primeiro.")
            return

        model_id = job_info['fine_tuned_model']
        print(f"TESTE DO MODELO FINE-TUNED: {model_id}")

        test_questions = [
            "Me fale sobre fones de ouvido Sony WH-1000XM4",
            "Quais as características do Samsung Galaxy S21 Ultra",
            "Descreva o MacBook Air com M1"
        ]

        for question in test_questions:
            print(f"\nPergunta: {question}")

            # Teste com modelo fine-tuned
            response_ft = client.chat.completions.create(
                model=model_id,
                messages=[
                    {"role": "system", "content": "Você é um assistente especializado em produtos da Amazon."},
                    {"role": "user", "content": question}
                ],
                max_tokens=150,
                temperature=0.7
            )

            answer_ft = response_ft.choices[0].message.content
            print(f"Resposta FINE-TUNED: {answer_ft}")

            # Comparar com modelo base
            response_base = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": "Você é um assistente de produtos."},
                    {"role": "user", "content": question}
                ],
                max_tokens=150,
                temperature=0.7
            )

            answer_base = response_base.choices[0].message.content
            print(f"Resposta BASE: {answer_base}")
            print("-" * 50)

    except Exception as e:
        print(f"Erro no teste: {e}")

In [6]:
# Executar teste (só funciona depois do fine-tuning completar)
test_finetuned_model()

TESTE DO MODELO FINE-TUNED: ft:gpt-3.5-turbo-1106:personal::CDX4FhCa

Pergunta: Me fale sobre fones de ouvido Sony WH-1000XM4
Resposta FINE-TUNED: Os fones de ouvido Sony WH-1000XM4 são conhecidos por oferecer uma excelente qualidade de som e cancelamento de ruído. Eles vêm com tecnologia de cancelamento de ruído adaptável, o que significa que eles podem ajustar automaticamente o nível de cancelamento de ruído com base no ambiente em que você está. Além disso, eles possuem uma longa duração da bateria, permitindo até 30 horas de reprodução com uma única carga. Os WH-1000XM4 também oferecem conectividade Bluetooth e suporte para assistentes de voz, como a Alexa da Amazon. Com um design confortável e controles intuitivos,
Resposta BASE: Os fones de ouvido Sony WH-1000XM4 são considerados uma das melhores opções de fones com cancelamento de ruído ativo disponíveis no mercado. Eles oferecem uma excelente qualidade de som, conforto e tecnologia avançada.

Além do cancelamento de ruído, os W

* 8: INTERFACE INTERATIVA COM GRADIO

In [7]:
def create_chatbot_interface():
    """Cria interface Gradio para o chatbot"""

    # Carregar modelo fine-tuned
    try:
        with open('/content/finetune_info.json', 'r') as f:
            job_info = json.load(f)

        if 'fine_tuned_model' in job_info:
            custom_model_id = job_info['fine_tuned_model']
            print(f"Modelo personalizado disponível: {custom_model_id}")
        else:
            custom_model_id = None
            print("Modelo personalizado não encontrado")

    except:
        custom_model_id = None
        print("Arquivo de modelo não encontrado")

    def generate_response(message, history, use_custom_model):
        """Gera resposta usando OpenAI"""

        # Escolher modelo baseado na seleção do usuário
        if use_custom_model and custom_model_id:
            model_id = custom_model_id
            system_message = "Você é um assistente especializado em produtos da Amazon. Forneça informações detalhadas e úteis sobre produtos baseado no seu treinamento específico."
        else:
            model_id = "gpt-3.5-turbo"
            system_message = "Você é um assistente de produtos. Forneça informações gerais sobre produtos."

        # Construir contexto da conversa
        messages = [{"role": "system", "content": system_message}]

        # Adicionar histórico
        for user_msg, assistant_msg in history:
            messages.append({"role": "user", "content": user_msg})
            messages.append({"role": "assistant", "content": assistant_msg})

        # Adicionar mensagem atual
        messages.append({"role": "user", "content": message})

        try:
            response = client.chat.completions.create(
                model=model_id,
                messages=messages,
                max_tokens=200,
                temperature=0.7
            )

            # Adicionar indicador do modelo usado na resposta
            model_indicator = " [PERSONALIZADO]" if use_custom_model and custom_model_id else " [PADRÃO]"
            answer = response.choices[0].message.content + model_indicator

            return answer

        except Exception as e:
            return f"Erro: {str(e)}"

    # Criar interface Gradio
    with gr.Blocks(title="Amazon Product Assistant") as demo:
        gr.HTML("<h1>Assistente de Produtos Amazon - Comparação de Modelos</h1>")

        # Seletor de modelo
        with gr.Row():
            model_selector = gr.Radio(
                choices=[
                    ("Modelo Personalizado (Fine-tuned)", True),
                    ("Modelo Padrão (GPT-3.5)", False)
                ],
                value=True if custom_model_id else False,
                label="Escolha o Modelo",
                interactive=True if custom_model_id else False
            )

        # Informações dos modelos
        if custom_model_id:
            gr.HTML(f"<p><strong>Modelo Personalizado:</strong> <code>{custom_model_id}</code></p>")
        else:
            gr.HTML("<p><strong>Aviso:</strong> Modelo personalizado não disponível. Usando apenas modelo padrão.</p>")

        gr.HTML("<p><strong>Modelo Padrão:</strong> <code>gpt-3.5-turbo</code></p>")

        chatbot = gr.Chatbot()

        with gr.Row():
            msg = gr.Textbox(
                placeholder="Faça uma pergunta sobre produtos...",
                label="Sua pergunta",
                scale=4
            )
            submit = gr.Button("Enviar", scale=1)

        clear = gr.Button("Limpar Chat")

        # Exemplos
        gr.Examples(
            examples=[
                "Me fale sobre fones de ouvido Sony",
                "Quais as características de smartphones Samsung?",
                "Descreva produtos Apple",
                "Preciso de informações sobre tênis Nike",
                "O que você sabe sobre panelas elétricas?"
            ],
            inputs=msg
        )

        # Informações sobre a comparação
        gr.HTML("""
        <div style="background-color: #f0f0f0; padding: 10px; margin-top: 20px; border-radius: 5px;">
        <h3>Como comparar os modelos:</h3>
        <ul>
        <li><strong>Modelo Personalizado:</strong> Treinado com produtos específicos da Amazon, deve dar respostas mais detalhadas e específicas</li>
        <li><strong>Modelo Padrão:</strong> Conhecimento geral, respostas mais genéricas sobre produtos</li>
        <li><strong>Indicador:</strong> Cada resposta mostra [PERSONALIZADO] ou [PADRÃO] para identificar qual modelo foi usado</li>
        </ul>
        </div>
        """)

        def respond(message, chat_history, use_custom):
            bot_message = generate_response(message, chat_history, use_custom)
            chat_history.append((message, bot_message))
            return "", chat_history

        def clear_chat():
            return []

        msg.submit(respond, [msg, chatbot, model_selector], [msg, chatbot])
        submit.click(respond, [msg, chatbot, model_selector], [msg, chatbot])
        clear.click(clear_chat, outputs=[chatbot])

    return demo

In [8]:
# Criar e executar interface
demo = create_chatbot_interface()
demo.launch(share=True, debug=True)

Modelo personalizado disponível: ft:gpt-3.5-turbo-1106:personal::CDX4FhCa


/tmp/ipython-input-3389249057.py:83: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://36f47bb63c91fed5df.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://36f47bb63c91fed5df.gradio.live
